Berechnung der Markdown

In [1]:
import numpy as np

### Hosseinkashi et al. 2024
##### Nachbauen der Ergebnisse
1. statistich signifikant

In [4]:
import numpy as np
import math # Import the math module for ln

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

# --- COEFFICIENTS FROM TABLE 3 (Combined Model, only statistically significant ORs) ---
# Odds Ratios (OR) were converted to GLM Coefficients (beta_i = ln(OR)).
# Intercepts (B0) are NOT provided and are ASSUMED to be 0 for all models.

GLM_COEFFICIENTS_V3 = {
    'Participation': {
        'Reliability Issues': math.log(0.13), # -2.040
        'Recurring': math.log(0.82),          # -0.199
        'ScreenShare': math.log(0.71),        # -0.342
        # WICHTIG: Die korrekte Bezeichnung aus der Tabelle
        'Small Meeting (8 Or Less)': math.log(7.13), # 1.964 
        'Short Call (10min. or less)': math.log(0.72),# -0.329
        'Headset': math.log(1.16),            # 0.148
        'Video Duration > 30%': math.log(1.17), # 0.157
        'B0': 0.0
    },
    
    'Inclusive': {
        'Quality Issues': math.log(0.35),      # -1.050
        'Reliability Issues': math.log(0.49),  # -0.713
        'Participation': math.log(4.05),       # 1.399
        # WICHTIG: Die korrekte Bezeichnung aus der Tabelle
        'Small Meeting (8 Or Less)': math.log(1.51), # 0.412
        'Short Call (10min. or less)': math.log(0.61),# -0.494
        'B0': 0.0
    },
    
    'Effective': {
        'Quality Issues': math.log(0.14),      # -1.966
        'Inclusive': math.log(45.48),          # 3.817
        'ScreenShare': math.log(1.39),         # 0.330
        # WICHTIG: Die korrekte Bezeichnung aus der Tabelle
        'Small Meeting (8 Or Less)': math.log(1.29), # 0.255
        'B0': 0.0
    }
}


def calculate_meeting_effectiveness_probability_v3(meeting_attributes: dict):
    """
    Calculates the probability of Effective using the *significant* GLM coefficients
    derived from Table 3 Odds Ratios, passing continuous probabilities.
    """
    
    # --- 0. Define all required root inputs ---
    required_root_inputs = set(
        list(GLM_COEFFICIENTS_V3['Participation'].keys()) +
        list(GLM_COEFFICIENTS_V3['Inclusive'].keys()) +
        list(GLM_COEFFICIENTS_V3['Effective'].keys())
    )
    # Entferne die berechneten Variablen und Intercepts
    required_root_inputs.difference_update(['B0', 'Participation', 'Inclusive']) 
    
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in the model.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    log_odds_part = GLM_COEFFICIENTS_V3['Participation']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Participation'].items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    log_odds_inclusive = GLM_COEFFICIENTS_V3['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation 
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Effective (P_effective) ---
    log_odds_effective = GLM_COEFFICIENTS_V3['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive 

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 65)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 65)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 65)
    
    return P_effective

# --- KORRIGIERTE SZENARIO-DEFINITIONEN (Fix des 'Or' vs 'or' Fehlers) ---

## SCENARIO 1: Ideal Meeting (Optimal)
ideal_meeting_attributes_v3 = {
    # Alle negativen Koeffizienten auf 0 setzen
    'Reliability Issues': 0, 
    'Quality Issues': 0,     
    'Recurring': 0,
    'ScreenShare': 0,
    'Short Call (10min. or less)': 0,
    
    # Alle positiven Koeffizienten auf 1 setzen
    'Small Meeting (8 Or Less)': 1, # FIX: Großes 'Or'
    'Headset': 1,
    'Video Duration > 30%': 1,
}

print("## Szenario 1: Optimales Meeting (Tabelle 3)")
probability_1_v3 = calculate_meeting_effectiveness_probability_v3(ideal_meeting_attributes_v3)
print(f"Final Probability 1: {probability_1_v3:.4f}")

print("\n" + "="*75 + "\n")

## SCENARIO 2: Poor Meeting (Schlecht)
poor_meeting_attributes_v3 = {
    # Alle negativen Koeffizienten auf 1 setzen
    'Reliability Issues': 1, 
    'Quality Issues': 1,     
    'Recurring': 1,
    'ScreenShare': 1,
    'Short Call (10min. or less)': 1,
    
    # Alle positiven Koeffizienten auf 0 setzen
    'Small Meeting (8 Or Less)': 0, # FIX: Großes 'Or'
    'Headset': 0,
    'Video Duration > 30%': 0,
}

print("## Szenario 2: Schlechtes Meeting (Tabelle 3)")
probability_2_v3 = calculate_meeting_effectiveness_probability_v3(poor_meeting_attributes_v3)
print(f"Final Probability 2: {probability_2_v3:.4f}")

## Szenario 1: Optimales Meeting (Tabelle 3)
--- Intermediate Results (Tabelle 3 Koeffizienten, nur signifikant) ---
Log-Odds Participation (BX_P): 0.0000
Predicted P(Participation): 0.5000
-----------------------------------------------------------------
Log-Odds Inclusive (BX_I): 0.0000
Predicted P(Inclusive): 0.5000
-----------------------------------------------------------------
Log-Odds Effective (BX_E): 0.2546
Final Predicted P(Effective): 0.5633
-----------------------------------------------------------------
Final Probability 1: 0.5633


## Szenario 2: Schlechtes Meeting (Tabelle 3)
ERROR: Missing essential root attributes: Small Meeting (8 Or Less)
Please provide values (0 or 1) for all root inputs defined in the model.


TypeError: unsupported format string passed to NoneType.__format__

2. nicht statistisch signifikant

In [5]:
import numpy as np

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

def or_to_beta(odds_ratio):
    """Converts an Odds Ratio (OR) to a GLM Coefficient (beta) using the natural logarithm."""
    return np.log(odds_ratio)

# --- COEFFICIENTS FROM TABLE 3 (Adjusted Odds Ratios -> converted to Beta) ---
# NOTE: Intercepts (B0) are NOT provided and are ASSUMED to be 0 for all models.

# Table 3 lists Odds Ratios for:
# Effective = f(Quality Issues, Inclusive, ScreenShare, Small Meeting)
# Inclusive = f(Quality Issues, Reliability Issues, Participation, Small Meeting, Short Call)
# Participation = f(Reliability Issues, Recurring, ScreenShare, Small Meeting, Short Call, Headset, Video Duration)

# 1. Model for Participation
# Participation = f(Reliability Issues, Recurring, ScreenShare, Small Meeting, Short Call, Headset, Video Duration > 30%)
COEFS_PARTICIPATION = {
    'Short Call (10min. or less)': or_to_beta(0.72), # Negative influence
    'B0': 0.0 # Assumed Intercept
}

# 2. Model for Inclusive
# Inclusive = f(Quality Issues, Reliability Issues, Participation, Small Meeting, Short Call)
COEFS_INCLUSIVE = {
    'Short Call (10min. or less)': or_to_beta(0.61), # Negative influence
    'B0': 0.0 # Assumed Intercept
}

# 3. Model for Effective (Final Outcome)
# Effective = f(Quality Issues, Inclusive, ScreenShare, Small Meeting)
COEFS_EFFECTIVE = {
    'B0': 0.0 # Assumed Intercept
}


def calculate_meeting_effectiveness_probability_v3(meeting_attributes: dict):
    """
    Calculates the probability of Effective using Adjusted Odds Ratios from Table 3,
    converted to GLM coefficients (beta_i = ln(OR)), and cascading probabilities.
    """
    
    # --- Define all required root inputs based on the union of all model inputs in Table 3 ---
    required_root_inputs = set(
        list(COEFS_PARTICIPATION.keys()) +
        ['Quality Issues'] + ['Reliability Issues'] +
        ['Short Call (10min. or less)'] + ['Headset'] + ['Video Duration > 30%'] +
        ['Recurring'] + ['ScreenShare']
    )
    required_root_inputs.discard('B0')
    required_root_inputs.discard('Participation') # These are intermediate outcomes
    required_root_inputs.discard('Inclusive')

    # Check for missing required inputs
    missing_inputs = [key for key in required_root_inputs if key not in meeting_attributes]
    if missing_inputs:
        print(f"ERROR: Missing essential root attributes: {', '.join(missing_inputs)}")
        print("Please provide values (0 or 1) for all root inputs defined in Table 3.")
        return None

    
    # --- 1. Calculate Participation (P_participation) ---
    log_odds_part = COEFS_PARTICIPATION['B0']
    for attr, coef in COEFS_PARTICIPATION.items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # --- 2. Calculate Inclusive (P_inclusive) ---
    log_odds_inclusive = COEFS_INCLUSIVE['B0']
    for attr, coef in COEFS_INCLUSIVE.items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation # Use continuous probability
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # --- 3. Calculate Effective (P_effective) ---
    log_odds_effective = COEFS_EFFECTIVE['B0']
    for attr, coef in COEFS_EFFECTIVE.items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive # Use continuous probability

    P_effective = inv_logit(log_odds_effective)

    # --- Print Intermediate Results (Continuous) ---
    print("--- Intermediate Results (Table 3 ORs -> Beta Coefficients) ---")
    print(f"Log-Odds Participation (BX_P): {log_odds_part:.4f}")
    print(f"Predicted P(Participation): {P_participation:.4f}")
    print("-" * 60)
    print(f"Log-Odds Inclusive (BX_I): {log_odds_inclusive:.4f}")
    print(f"Predicted P(Inclusive): {P_inclusive:.4f}")
    print("-" * 60)
    print(f"Log-Odds Effective (BX_E): {log_odds_effective:.4f}")
    print(f"Final Predicted P(Effective): {P_effective:.4f}")
    print("-" * 60)
    
    return P_effective


# --- SCENARIOS DEFINED FOR TABLE 3 VARIABLES ---

# Note on New Variables:
# - 'Small Meeting (8 or less)' is POSITIVE (OR=7.13 to Participation)
# - 'Quality Issues', 'Reliability Issues' are NEGATIVE (OR < 1)

## SCENARIO 1: Ideal Meeting (Maximize Positive Inputs)
ideal_meeting_attributes_v3 = {
    # Maximize Participation
    'Reliability Issues': 0,        # No Issues (OR 0.13 is negative)
    'Recurring': 0,                 # Not Recurring (OR 0.82 is negative)
    'ScreenShare': 0,               # No Screen Share (OR 0.71 is negative)
    'Small Meeting (8 or less)': 1, # Yes, Small Meeting (OR 7.13 is strong positive)
    'Short Call (10min. or less)': 0, # Not Short Call (OR 0.72 is negative)
    'Headset': 1,                   # Yes, Headset (OR 1.16 is positive)
    'Video Duration > 30%': 1,      # Yes, long video (OR 1.17 is positive)
    
    # Maximize Effective (and Inclusive)
    'Quality Issues': 0,            # No Issues (OR 0.14 is strong negative)
}

print("## Scenario 1: High-Potential Meeting (Using Table 3 Adjusted ORs)")
probability_1_v3 = calculate_meeting_effectiveness_probability_v3(ideal_meeting_attributes_v3)
print(f"Final Probability 1: {probability_1_v3:.4f}")

print("\n" + "="*70 + "\n")

## SCENARIO 2: Poor Meeting (Maximize Negative Inputs)
poor_meeting_attributes_v3 = {
    # Minimize Participation
    'Reliability Issues': 1,        # Yes, Reliability Issues (OR 0.13 is negative)
    'Recurring': 1,                 # Yes, Recurring (OR 0.82 is negative)
    'ScreenShare': 1,               # Yes, Screen Share (OR 0.71 is negative)
    'Small Meeting (8 or less)': 0, # No, Large Meeting (OR 7.13 is strong positive)
    'Short Call (10min. or less)': 1, # Yes, Short Call (OR 0.72 is negative)
    'Headset': 0,                   # No Headset (OR 1.16 is positive)
    'Video Duration > 30%': 0,      # No, short video (OR 1.17 is positive)
    
    # Minimize Effective (and Inclusive)
    'Quality Issues': 1,            # Yes, Quality Issues (OR 0.14 is strong negative)
}

print("## Scenario 2: Low-Potential Meeting (Using Table 3 Adjusted ORs)")
probability_2_v3 = calculate_meeting_effectiveness_probability_v3(poor_meeting_attributes_v3)
print(f"Final Probability 2: {probability_2_v3:.4f}")

## Scenario 1: High-Potential Meeting (Using Table 3 Adjusted ORs)
--- Intermediate Results (Table 3 ORs -> Beta Coefficients) ---
Log-Odds Participation (BX_P): 0.0000
Predicted P(Participation): 0.5000
------------------------------------------------------------
Log-Odds Inclusive (BX_I): 0.0000
Predicted P(Inclusive): 0.5000
------------------------------------------------------------
Log-Odds Effective (BX_E): 0.0000
Final Predicted P(Effective): 0.5000
------------------------------------------------------------
Final Probability 1: 0.5000


## Scenario 2: Low-Potential Meeting (Using Table 3 Adjusted ORs)
--- Intermediate Results (Table 3 ORs -> Beta Coefficients) ---
Log-Odds Participation (BX_P): -0.3285
Predicted P(Participation): 0.4186
------------------------------------------------------------
Log-Odds Inclusive (BX_I): -0.4943
Predicted P(Inclusive): 0.3789
------------------------------------------------------------
Log-Odds Effective (BX_E): 0.0000
Final Predicted P(Eff

In [8]:
import numpy as np
import math

# --- FUNKTIONEN UND KOEFFIZIENTEN (Basierend auf Tabelle 3, nur signifikant) ---

def inv_logit(log_odds):
    """Calculates the probability (inverse logit function: P = 1 / (1 + e^(-LogOdds)))."""
    return 1 / (1 + np.exp(-log_odds))

# GLM COEFFICIENTS (Odds Ratios aus Tabelle 3 umgerechnet zu ln(OR) und auf Signifikanz gefiltert)
GLM_COEFFICIENTS_V3 = {
    # Partizipation: Beta-Werte (ln(OR))
    'Participation': {
        'Small Meeting (8 Or Less)': math.log(7.13), # 1.964
        'B0': 0.0
    },
    # Inklusiv: Beta-Werte (ln(OR))
    'Inclusive': {
        'Small Meeting (8 Or Less)': math.log(1.51), # 0.412
        'B0': 0.0
    },
    # Effektiv: Beta-Werte (ln(OR))
    'Effective': {
        'Small Meeting (8 Or Less)': math.log(1.29), # 0.255
        'B0': 0.0
    }
}

# Mapping für lesbare Ausgaben und Einfluss-Interpretation
COEFFICIENT_DESCRIPTIONS = {
    'Reliability Issues': 'Zuverlässigkeitsprobleme (stark negativ)',
    'Recurring': 'Wiederkehrendes Meeting (leicht negativ)',
    'ScreenShare': 'Bildschirmfreigabe (negativ)',
    'Small Meeting (8 Or Less)': 'Kleine Gruppe (<= 8 Personen, stark positiv)',
    'Short Call (10min. or less)': 'Kurzes Meeting (<= 10 Min, negativ)',
    'Headset': 'Headset benutzt (positiv)',
    'Video Duration > 30%': 'Lange Videonutzung (>30%, positiv)',
    'Quality Issues': 'Qualitätsprobleme (sehr stark negativ)',
}


def calculate_effectiveness_metrics(meeting_attributes):
    """Führt die kaskadierte Berechnung durch."""
    
    # 1. Participation
    log_odds_part = GLM_COEFFICIENTS_V3['Participation']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Participation'].items():
        if attr in meeting_attributes:
            log_odds_part += coef * meeting_attributes[attr]
            
    P_participation = inv_logit(log_odds_part)
    
    # 2. Inclusive
    log_odds_inclusive = GLM_COEFFICIENTS_V3['Inclusive']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Inclusive'].items():
        if attr in meeting_attributes:
            log_odds_inclusive += coef * meeting_attributes[attr]
        elif attr == 'Participation':
            log_odds_inclusive += coef * P_participation 
    
    P_inclusive = inv_logit(log_odds_inclusive)

    
    # 3. Effective
    log_odds_effective = GLM_COEFFICIENTS_V3['Effective']['B0']
    for attr, coef in GLM_COEFFICIENTS_V3['Effective'].items():
        if attr in meeting_attributes:
            log_odds_effective += coef * meeting_attributes[attr]
        elif attr == 'Inclusive':
            log_odds_effective += coef * P_inclusive 

    P_effective = inv_logit(log_odds_effective)
    
    return {
        'P_participation': P_participation, 'P_inclusive': P_inclusive, 'P_effective': P_effective,
        'log_odds_part': log_odds_part, 'log_odds_inclusive': log_odds_inclusive, 'log_odds_effective': log_odds_effective,
    }


def generate_effectiveness_report(scenario_name: str, meeting_attributes: dict) -> str:
    """
    Generiert einen Markdown-formatierten Bericht, der direkt exportiert werden kann.
    """
    
    try:
        metrics = calculate_effectiveness_metrics(meeting_attributes)
    except Exception as e:
        return f"Fehler bei der Berechnung für Szenario {scenario_name}: {e}"
    
    report = []
    
    # --- I. BERICHTSÜBERSCHRIFT ---
    report.append(f"### Meeting-Effizienz-Analyse: {scenario_name}")
    report.append("-" * 50)
    
    # --- II. ERGEBNIS-ZUSAMMENFASSUNG ---
    report.append("#### Zusammenfassung")
    report.append(f"Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **{metrics['P_effective']:.2%}**")
    report.append(f"*(Log-Odds (BX): {metrics['log_odds_effective']:.4f})*")
    report.append("\n")
    
    return "\n".join(report)

# --- ANWENDUNG DER SZEANRIEN ---

# Scenario 1 (Optimal) Attributes
ideal_meeting_attributes_v3 = {
    'Small Meeting (8 Or Less)': 1
}

# Scenario 2 (Schlecht) Attributes
poor_meeting_attributes_v3 = {
    'Small Meeting (8 Or Less)': 0
}


# 1. Bericht für Szenario 1 generieren
report_s1 = generate_effectiveness_report("Optimales Meeting (S1)", ideal_meeting_attributes_v3)
print(report_s1)

print("\n" + "="*75 + "\n")

# 2. Bericht für Szenario 2 generieren
report_s2 = generate_effectiveness_report("Schlechtes Meeting (S2)", poor_meeting_attributes_v3)
print(report_s2)

### Meeting-Effizienz-Analyse: Optimales Meeting (S1)
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **56.33%**
*(Log-Odds (BX): 0.2546)*




### Meeting-Effizienz-Analyse: Schlechtes Meeting (S2)
--------------------------------------------------
#### Zusammenfassung
Die berechnete Wahrscheinlichkeit, dass das Meeting **effektiv** ist: **50.00%**
*(Log-Odds (BX): 0.0000)*


